In [ ]:
# ==== 单Cell：EEG Status 事件浏览（只一张图，松开滑块才刷新） ====
%matplotlib inline

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import mne

# -------- 配置 --------
DATA_DIR = Path(r"C:\Users\huiwe\Desktop\BME Lab\new_RSVP\rsvp_mne_files\rsvp_mne_files")
FILE_PATTERN = "rsvp_*Hz_*[ab].raw.fif"
STIM_CHANNEL = "Status"
# ---------------------

# 1) 列出文件
files = sorted(DATA_DIR.glob(FILE_PATTERN))
if not files:
    raise SystemExit(f"No files found in {DATA_DIR} matching {FILE_PATTERN}")

file_dd = widgets.Dropdown(
    options=[(p.name, p) for p in files],
    description="文件:",
    layout=widgets.Layout(width="600px"),
)

# 关键：continuous_update=False → 只在“松开”时触发
win = widgets.FloatRangeSlider(
    value=[0.0, 30.0], min=0.0, max=30.0, step=0.5,
    description="时间窗口 (s):", readout=True, continuous_update=False,
    layout=widgets.Layout(width="600px"),
)

show_burst = widgets.Checkbox(value=True, description="显示 Burst 开始/结束")
show_non   = widgets.Checkbox(value=True, description="显示 Non-target (1002)")
show_tar   = widgets.Checkbox(value=True, description="显示 Target (1..640)")

# 2) 读取并缓存事件（避免重复IO）
_cache = {}  # fpath -> (times, codes, sfreq, duration)
def load_events(fpath: Path):
    if fpath not in _cache:
        raw = mne.io.read_raw_fif(fpath, preload=False, verbose=False)
        sfreq = float(raw.info["sfreq"])
        events = mne.find_events(raw, stim_channel=STIM_CHANNEL,
                                 output="onset", shortest_event=1, verbose=False)
        times = events[:, 0] / sfreq
        codes = events[:, 2].astype(int)
        duration = float(raw.times[-1])
        _cache[fpath] = (times, codes, sfreq, duration)
    return _cache[fpath]

# 3) 单一输出容器：每次刷新前先 clear → 永远只有一张图
out = widgets.Output()

def redraw(*_):
    with out:
        out.clear_output(wait=True)           # 覆盖旧图，不累加
        fpath = file_dd.value
        (times, codes, sfreq, duration) = load_events(fpath)

        tmin, tmax = map(float, win.value)
        tmin = max(0.0, tmin); tmax = min(duration, tmax)
        if tmax <= tmin: tmax = min(duration, tmin + 1.0)
        span = tmax - tmin                     # 窗口时长（会显示在标题里）

        inwin = (times >= tmin) & (times <= tmax)
        burst_start = (codes == 1000) & inwin
        burst_end   = (codes == 1001) & inwin
        non_target  = (codes == 1002) & inwin
        target      = (codes <  1000) & inwin  # 1..640

        fig, ax = plt.subplots(figsize=(10, 4), dpi=110)
        if show_burst.value:
            ax.scatter(times[burst_start], codes[burst_start], marker="|", s=220, color="red",     label="Burst start (1000)")
            ax.scatter(times[burst_end],   codes[burst_end],   marker="|", s=220, color="darkred", label="Burst end (1001)")
        if show_non.value:
            ax.scatter(times[non_target],  codes[non_target],  marker="|", s=140, color="gray", alpha=0.6, label="Non-target (1002)")
        if show_tar.value:
            ax.scatter(times[target],      codes[target],      marker="|", s=160, color="blue",          label="Target (1..640)")

        ax.set_xlim(tmin, tmax)
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Event code")
        # 这里把“总时长”和“窗口时长/范围”都放进标题，随着滑块变化
        ax.set_title(f"{fpath.name} — sfreq={sfreq:.1f}Hz, full={duration:.1f}s | "
                     f"window={tmin:.1f}-{tmax:.1f}s (span={span:.1f}s)")
        ax.grid(True, alpha=0.3)
        ax.legend(loc="upper right")
        plt.tight_layout()
        plt.show()

# 4) 切换文件时更新滑块范围并重画
def on_file_change(change):
    if change["name"] == "value" and change["new"] is not None:
        _, _, _, dur = load_events(change["new"])
        win.min = 0.0
        win.max = round(dur, 2)
        win.value = (0.0, min(30.0, dur))     # 初始窗口
        redraw()

file_dd.observe(on_file_change, names="value")
win.observe(redraw, names="value")            # 松手才触发，因为 continuous_update=False
show_burst.observe(redraw, names="value")
show_non.observe(redraw, names="value")
show_tar.observe(redraw, names="value")

# 5) 布局与首次绘制（只显示一次 UI + 一个 out）
ui = widgets.VBox([file_dd, win, widgets.HBox([show_burst, show_non, show_tar])])
display(ui, out)

# 初始化
on_file_change({"name":"value","new":file_dd.value})


Output()